In [1]:
import pandas as pd
from gensim import utils
import json
import csv
from smart_open import open
from hashlib import sha1
from datasketch import MinHash, MinHashLSH
from tqdm import tqdm
import pickle
tqdm.pandas()

In [2]:
from spacy.lang.ru import Russian
nlp = Russian()
nlp.add_pipe('sentencizer')

In [3]:
def process(section):
    res = []
    doc = nlp(section)
    for sent in doc.sents:
        sent = str(sent)
        x = sent.replace('\n', '')
        if x:
            res.append([x])
    return res

In [15]:
%%time

i = 0

with open('ruwiki-latest.json.gz', 'rb') as f:
        for line in f:
            i += 1

print(f'Total articles  -- {i}')

Total articles  -- 2801969
Wall time: 1min 25s


In [20]:
# %%time

# import csv
# i = 0
# cnt = 0
# file_name = 'texts0.csv'
# cs = open(file_name, 'w', newline='', encoding='utf-8')
# linewriter = csv.writer(cs)

# tmp = []

# with open('ruwiki-latest.json.gz', 'rb') as f:

#         for line in f:
#             i += 1
#             article = json.loads(line)

#             for section_text in article['section_texts']:
#                 art = process(section_text)
#                 tmp.extend(art)
#                 tmp.append(['EOS'])        

#             if i == 466995:
#                 linewriter.writerows(tmp)
#                 tmp = []
#                 cs.close() 
#                 i = 0          
#                 cnt += 1
#                 print('Starting new file')
#                 file_name = 'texts' + str(cnt) + '.csv'
#                 cs = open(file_name, 'w', newline='', encoding='utf-8')
#                 linewriter = csv.writer(cs)
            
#             elif i % 100000 == 0:
#                 print(f'Current i -- {i}')
#             if len(tmp) > 100:
#                 linewriter.writerows(tmp)
#                 tmp = []

# if tmp:
#     linewriter.writerows(tmp) 
#     tmp = []
#     cs.close()

Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Starting new file
Current i -- 0
Current i -- 100000
Current i -- 200000
Current i -- 300000
Current i -- 400000
Wall time: 1h 6min 53s


In [8]:
def hashes_simple(x):
    m1 = MinHash(num_perm=16)
    x = x.lower()
    _ = [m1.update(token.encode('utf8')) for token in x.split()]
    return m1

In [5]:
def process_txt(x):
    if x == '\n':
        return x
    else:
        x = x.replace('\n', '')
        return x

In [6]:
df = pd.read_csv('texts0.csv', header=None, names = ['Text'],
                             encoding='utf-8', low_memory=True, nrows=50)

# Upload hashes in memory (csv)

In [5]:
%%time

lsh = MinHashLSH(threshold=0.8, num_perm=16)

comp = 1000000
i = 0
with lsh.insertion_session() as session:
    for chunk in pd.read_csv('texts5.csv', header=None, names = ['Text'],
                             encoding='utf-8', low_memory=True, chunksize=500000):
        chunk = chunk.dropna()
        chunk = chunk[chunk['Text'] != 'EOS']
        texts = chunk.Text.progress_apply(hashes_simple)
        texts = texts.to_list()
        hashes = [('m'+str(u), x) for x, u in zip(texts, range(i, i+len(texts)+1))]
        i += len(texts)
        if i > comp:
            print(f'Processed sentences -- {i:,}')
            comp += 1000000
        for key, minhash in hashes:
            session.insert(key, minhash)

In [4]:
print('All hashes are added to the LSH')

All hashes are added to the LSH


In [5]:
print(f'Total number of sentences in file -- {i:,}')

Total number of sentences in file -- 6,449,430


In [6]:
del texts
del chunk
del hashes
del i
del comp

# Upload hashed into memory (txt)

In [4]:
lsh = MinHashLSH(threshold=0.8, num_perm=16)

In [6]:
def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

In [30]:
%%time

lsh = MinHashLSH(threshold=0.8, num_perm=16)

comp = 1000000
i = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with lsh.insertion_session() as session:
    with open('Ready\wiki4.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1):
            sents = [process_txt(y) for y in piece if y != '\n']
            texts = [hashes_simple(y) for y in tqdm(sents)]
            hashes = [('m'+str(u), y) for y, u in zip(texts, range(i, i+len(texts)+1))]
            i += len(texts)
            if i > comp:
                print(f'Processed sentences -- {i:,}')
                comp += 1000000
            for key, minhash in hashes:
                session.insert(key, minhash)

100%|████████████████████████████████████████████████████████████████████████| 359400/359400 [02:27<00:00, 2441.67it/s]


Processed sentences -- 1,091,290


100%|████████████████████████████████████████████████████████████████████████| 369602/369602 [02:29<00:00, 2474.61it/s]


Processed sentences -- 2,190,066


100%|████████████████████████████████████████████████████████████████████████| 370327/370327 [02:32<00:00, 2424.29it/s]


Processed sentences -- 3,290,484


100%|████████████████████████████████████████████████████████████████████████| 355620/355620 [02:26<00:00, 2422.25it/s]


Processed sentences -- 4,343,797


100%|████████████████████████████████████████████████████████████████████████| 315261/315261 [02:11<00:00, 2398.55it/s]


Wall time: 32min 46s


In [25]:
del texts
del piece
del hashes
del i
del comp

In [31]:
%%time
pickle.dump(lsh, open( "save4.p", "wb" ))

Wall time: 35.8 s


In [4]:
%%time
w0 = pickle.load(open("Wiki_pickle/save.p", "rb"))
print('w0 done')
w1 = pickle.load(open("Wiki_pickle/save1.p", "rb"))
print('w1 done')
w2 = pickle.load(open("Wiki_pickle/save2.p", "rb"))
print('w2 done')
w3 = pickle.load(open("Wiki_pickle/save3.p", "rb"))
print('w3 done')
w4 = pickle.load(open("Wiki_pickle/save4.p", "rb"))
print('w4 done')
w5 = pickle.load(open("Wiki_pickle/save5.p", "rb"))
print('w5 done')

w0 done
w1 done
w2 done
w3 done
w4 done
w5 done
Wall time: 20min 28s


# csv cleaning

In [2]:
from bs4 import BeautifulSoup
from html import unescape
import unicodedata
import string
import unidecode

def escape_html(text):
    soup = BeautifulSoup(unescape(text), 'html.parser')
    text = soup.text
    text = unicodedata.normalize('NFKC', text)
    return text

def my_replace(match):
    match = match.group()
    return unidecode.unidecode(match)

def replace_dashes(text):
    dashes = '-‐‑‒–—―'
    for dash in dashes:
        text = text.replace(dash, '-')
    text = text.replace('--', '-').replace('--', '-')
    return text

def replace_quotes(text):
    quotes = '»«”“„‟❝❞〝〞〟＂'
    for q in quotes:
        if isinstance(text, str):
            text = text.replace(q, '"')
    if isinstance(text, str):
        text = text.replace(',,', '"')
    text = text.replace("<<", '"')
    text = text.replace(">>", '"')
    return text

In [10]:
def process_new(section):
    res = []
    doc = nlp(section)
    for line in doc.sents:
        line = str(line)
        line = line.replace('\n', '')
        if line:
            line = escape_html(line).replace('  ', ' ').replace('  ', ' ')
            line = re.sub('([^А-яЁё])', my_replace, line)
            line = replace_dashes(line)
            line = replace_quotes(line)
            line = line.replace('\r', '')
            if line:
                line = line.replace('  ', ' ').replace('  ', ' ')
                res.append(line)
    return res

In [13]:
%%time
import re 

d = 0
with open('geo_wac0_v01.txt', 'w', encoding='utf-8', newline='\n') as f:
    for chunk in pd.read_csv('geo_wac0.csv', header=None, names = ['Text'], encoding='utf-8', chunksize=10000):
        df_geo = chunk.Text.to_list()

        for text in tqdm(df_geo):
            sents = process_new(text)
            hashes = [hashes_simple(x) for x in sents]
            res = []
            for i in range(len(hashes)):
                key = hashes[i]
                if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0 and len(w3.query(key)) == 0 and len(w4.query(key)) == 0 \
                and len(w5.query(key)) == 0:
                    y = sents[i] + '\n'
                    res.append(y)
                else:
                    d += 1
                    if d % 100000 == 0:
                        print(f'Duplicates found --- {d:,}')
            if res:
                for s in res:
                    f.write(s)
                f.write('\n')

 45%|█████████████████████████████████▉                                          | 4464/10000 [00:49<00:46, 120.01it/s]c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
 99%|███████████████████████████████████████████████████████████████████████████▎| 9916/10000 [00:27<00:00, 352.84it/s]c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: ". . ." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
 57%|███████████████████████████████████████████▋                                | 5743/10000 [00:14<00:13, 319.32it/s]c:\users\администратор\appdata\local\programs\python\python36\lib\site-pa

100%|███████████████████████████████████████████████████████████████████████████| 10000/10000 [00:22<00:00, 450.40it/s]


 13%|█████████▌                                                                  | 1255/10000 [00:02<00:20, 436.97it/s]

Duplicates found --- 100,000


 72%|██████████████████████████████████████████████████████▌                     | 7180/10000 [00:19<00:07, 369.08it/s]


KeyboardInterrupt: 

In [14]:
print(f'Total duplicates in file --- {d:,}')

Total duplicates in file --- 1,188,319


In [14]:
del w0
del w1
del w2
del w3
del w4
del w5

# txt cleaning

In [3]:
%%time
w0 = pickle.load(open("rnc_pickle/pre-sov.p", "rb"))
print('w0 done')
w1 = pickle.load(open("rnc_pickle/sov.p", "rb"))
print('w1 done')
w2 = pickle.load(open("rnc_pickle/post-sov.p", "rb"))
print('w2 done')

w0 done
w1 done
w2 done
Wall time: 1min 49s


In [44]:
def hashes_simple_1(x):
    if x == 'PLACEHOLDER':
        return 'PLACEHOLDER'
    else:
        m1 = MinHash(num_perm=16)
        x = x.lower()
        _ = [m1.update(token.encode('utf8')) for token in x.split()]
        return m1

In [34]:
import re
from bs4 import BeautifulSoup
from html import unescape
import unicodedata
import string
import unidecode

def escape_html(text):
    soup = BeautifulSoup(unescape(text), 'html.parser')
    text = soup.text
    text = unicodedata.normalize('NFKC', text)
    return text

def my_replace(match):
    match = match.group()
    return unidecode.unidecode(match)

def replace_dashes(text):
    dashes = '-‐‑‒–—―'
    for dash in dashes:
        text = text.replace(dash, '-')
    text = text.replace('--', '-').replace('--', '-')
    return text

def replace_quotes(text):
    quotes = '»«”“„‟❝❞〝〞〟＂'
    for q in quotes:
        if isinstance(text, str):
            text = text.replace(q, '"')
    if isinstance(text, str):
        text = text.replace(',,', '"')
    text = text.replace("<<", '"')
    text = text.replace(">>", '"')
    text = text.replace(">", '')
    text = text.replace("<", '')
    return text

def process_txt_new(line):
    if line == '\n':
        return 'PLACEHOLDER'
    else:
        line = line.replace('\n', '')
        if len(line):
            line = escape_html(line).replace('  ', ' ').replace('  ', ' ')
            line = re.sub('([^А-яЁё])', my_replace, line)
            line = replace_dashes(line)
            line = replace_quotes(line)
            line = line.replace('\r', '')
            if len(line):
                line = line.replace('  ', ' ').replace('  ', ' ') 
                if len(line):
                    return line

In [56]:
%%time

d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac0_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac0_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 126496/126496 [00:37<00:00, 3397.67it/s]


100%|████████████████████████████████████████████████████████████████████████| 115921/115921 [00:33<00:00, 3414.21it/s]


Duplicates found --- 100,000


100%|██████████████████████████████████████████████████████████████████████████| 91929/91929 [00:26<00:00, 3461.45it/s]


Wall time: 2h 10min 1s


In [57]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 135,580


In [58]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac1_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac1_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 94916/94916 [00:33<00:00, 2873.62it/s]


100%|████████████████████████████████████████████████████████████████████████| 135354/135354 [00:42<00:00, 3155.57it/s]


Duplicates found --- 100,000


100%|████████████████████████████████████████████████████████████████████████| 124133/124133 [00:37<00:00, 3353.81it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "C:" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|████████████████████████████████████████████████████████████████████████| 151754/151754 [00:44<00:00, 3444.04it/s]


100%|████████████████████████████████████████████████████████████████████████| 124655/124655 [00:36<00:00, 3391.10it/s]


Duplicates found --- 200,000


100%|████████████████████████████████████████████████████████████████████████| 136630/136630 [00:41<00:00, 3323.13it/s]


Duplicates found --- 300,000


100%|████████████████████████████████████████████████████████████████████████████| 8631/8631 [00:02<00:00, 3326.06it/s]


In [59]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 325,460


In [60]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac2_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac2_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 110802/110802 [00:33<00:00, 3348.73it/s]


Duplicates found --- 100,000


100%|████████████████████████████████████████████████████████████████████████| 120466/120466 [00:35<00:00, 3385.99it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "D:" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|██████████████████████████████████████████████████████████████████████████| 89937/89937 [00:30<00:00, 2907.94it/s]


100%|████████████████████████████████████████████████████████████████████████| 109736/109736 [00:32<00:00, 3341.44it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "..... ." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|████████████████████████████████████████████████████████████████████████| 142025/142025 [00:41<00:00, 3383.61it/s]


Duplicates found --- 200,000


100%|██████████████████████████████████████████████████████████████████████████| 88150/88150 [00:25<00:00, 3438.04it/s]


In [61]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 260,719


In [62]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac3_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac3_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 119862/119862 [00:34<00:00, 3440.69it/s]


100%|████████████████████████████████████████████████████████████████████████| 120299/120299 [00:33<00:00, 3550.95it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: ". ...." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|████████████████████████████████████████████████████████████████████████| 123189/123189 [00:35<00:00, 3435.65it/s]


Duplicates found --- 100,000


100%|████████████████████████████████████████████████████████████████████████████| 5537/5537 [00:01<00:00, 3581.38it/s]


In [63]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 139,630


In [64]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac4_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac4_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|██████████████████████████████████████████████████████████████████████████| 81198/81198 [00:27<00:00, 2985.42it/s]


100%|████████████████████████████████████████████████████████████████████████| 118898/118898 [00:36<00:00, 3225.72it/s]


100%|████████████████████████████████████████████████████████████████████████| 119583/119583 [00:35<00:00, 3410.10it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: ".... ." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|████████████████████████████████████████████████████████████████████████| 111919/111919 [00:33<00:00, 3335.28it/s]


Duplicates found --- 100,000


100%|██████████████████████████████████████████████████████████████████████████| 56928/56928 [00:17<00:00, 3218.60it/s]


In [65]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 133,778


In [66]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac5_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac5_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 105448/105448 [00:35<00:00, 2968.20it/s]
c:\users\администратор\appdata\local\programs\python\python36\lib\site-packages\bs4\__init__.py:336: MarkupResemblesLocatorWarning: "Ready." looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  MarkupResemblesLocatorWarning
100%|████████████████████████████████████████████████████████████████████████| 115563/115563 [00:34<00:00, 3391.24it/s]


100%|████████████████████████████████████████████████████████████████████████| 137866/137866 [00:40<00:00, 3413.58it/s]


100%|████████████████████████████████████████████████████████████████████████| 121499/121499 [00:36<00:00, 3326.76it/s]


Duplicates found --- 100,000


100%|████████████████████████████████████████████████████████████████████████| 107279/107279 [00:31<00:00, 3370.15it/s]


In [67]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 144,961


In [68]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac6_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac6_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 120342/120342 [00:39<00:00, 3060.54it/s]


100%|████████████████████████████████████████████████████████████████████████| 152674/152674 [00:44<00:00, 3433.56it/s]


Duplicates found --- 100,000


100%|████████████████████████████████████████████████████████████████████████| 141008/141008 [00:45<00:00, 3125.74it/s]


100%|██████████████████████████████████████████████████████████████████████████| 47890/47890 [00:14<00:00, 3215.47it/s]


In [69]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 189,280


In [70]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac7_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac7_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 111353/111353 [00:32<00:00, 3455.60it/s]


100%|████████████████████████████████████████████████████████████████████████| 127012/127012 [00:36<00:00, 3503.49it/s]


100%|████████████████████████████████████████████████████████████████████████| 106212/106212 [00:32<00:00, 3256.88it/s]


Duplicates found --- 100,000


100%|██████████████████████████████████████████████████████████████████████████| 33705/33705 [00:10<00:00, 3307.39it/s]


In [71]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 174,255


In [72]:
d = 0

def read_in_chunks_lines(file_object, chunk_size=40000000):
    while True:
        data = file_object.readlines(chunk_size)
        if not data:
            break
        yield data

with open('geo_wac8_vv_03.txt', 'w', encoding='utf-8', newline='\n') as f:
    with open('geo_onlywiki/geo_wac8_vv_02.txt', encoding='utf-8') as f1:
        for piece in read_in_chunks_lines(f1, chunk_size=10000000):
            geo_sents = [process_txt_new(x) for x in piece]
            geo_sents = [line for line in geo_sents if line is not None]
            geo_hash = [hashes_simple_1(x) for x in tqdm(geo_sents)]
            for i in range(len(geo_hash)):
                if geo_sents[i] == 'PLACEHOLDER':
                    f.write('\n')
                else:
                    key = geo_hash[i]
                    if len(w0.query(key)) == 0 and len(w1.query(key)) == 0 and \
                len(w2.query(key)) == 0:
                        f.write(geo_sents[i] + '\n')
                    else:
                        d += 1
                        if d % 100000 == 0:
                            print(f'Duplicates found --- {d:,}')

100%|████████████████████████████████████████████████████████████████████████| 101313/101313 [00:30<00:00, 3331.39it/s]


100%|████████████████████████████████████████████████████████████████████████| 107591/107591 [00:33<00:00, 3166.17it/s]


Duplicates found --- 100,000


100%|██████████████████████████████████████████████████████████████████████████| 51555/51555 [00:16<00:00, 3075.79it/s]


In [73]:
print(f'Total duplicates in the file -- {d:,}')

Total duplicates in the file -- 104,912
